In [1]:
import imageio
imageio.plugins.ffmpeg.download()

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [12]:
#Canny Edge Detection
def getEdges(gray_img):
    # Define Canny Parameters
    kernel = 3
    lo = 60
    hi = 120
        
    gray_blur = cv2.GaussianBlur(gray_img,(kernel,kernel),0)
    edges = cv2.Canny(gray_blur,lo,hi)
    
    return edges

def getLines(mask_img,edge_img):
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = 1* np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15   # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 8 #minimum number of pixels making up a line
    max_line_gap = 10  # maximum gap in pixels between connectable line segments
    
    
    color_edges = np.dstack((edge_img, edge_img, edge_img)) 
    line_image = np.copy(mask_img)*0 # creating a blank to draw lines on
    line_image = np.dstack((line_image,line_image,line_image))
    
    lines = cv2.HoughLinesP(mask_img, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
    
    line_image = findMainLines(line_image,lines)
    return line_image


def avgPoints(points):
    
    ret = [0 for i in range(2)]
    for i in range(2):
        ret[i] = sum(point[i] for point in points)
    
    for i in range(2):
        ret[i] /= len(points)
    return ret

def calcYIntercept(m,point):
    return point[1] - point[0]*m

def calcX(y,m,b):
    return int((y-b)/m)

def calcEndpoints(m,point,y1,y2):
    b = calcYIntercept(m,point)
    #print("B: ",b)
    x1 = calcX(y1,m,b)
    x2 = calcX(y2,m,b)
    return [x1,y1,x2,y2]
    

def findMainLines(line_image,lines):
    
    left_slopes = []
    left_centers = []
    right_slopes = []
    right_centers = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x1-x2 != 0:
                slope = (y1-y2)/(x1-x2)
                center = [(x1+x2)/2,(y1+y2)/2]
                abs_slope = abs(slope)
                if abs_slope > 3/7 and abs_slope < 6/7:
                    if slope == abs_slope:
                        left_slopes.append(slope)
                        left_centers.append(center)
                        
                    else:
                        right_slopes.append(slope)
                        right_centers.append(center)
                        
    #Left Endpoints
    lslope = sum(left_slopes)/len(left_slopes)
    lcenter = avgPoints(left_centers)
    rslope = sum(right_slopes)/len(right_slopes)
    rcenter = avgPoints(right_centers)
    le = calcEndpoints(lslope,lcenter,325,720)
    re = calcEndpoints(rslope,rcenter,325,720)
    
    #print(lslope,lcenter,le,rslope,rcenter,re)
    cv2.line(line_image,(le[0],le[1]),(le[2],le[3]),(2,179,228),10)
    cv2.line(line_image,(re[0],re[1]),(re[2],re[3]),(2,179,228),10)
    #cv2.line(line_image,(300,300),(400,400),(255,0,0),10)
    #print("Slopes:",sum(right_slopes)/len(right_slopes),sum(left_slopes)/len(left_slopes))
    #print("Centers:",avgPoints(right_centers),avgPoints(left_centers))
    return line_image

def overlayLines(img,line_image):
    return cv2.addWeighted(img,0.8,line_image,1,0)
    
def maskImg(img,vertices):
    
    ignore_mask_color = 255
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, np.int32(vertices), ignore_mask_color)
    masked_img = cv2.bitwise_and(img, mask)
    
    return masked_img

#Convert Grayscale Image To Be Compatable With RGB
def grayToThreeChannel(img):
    shape = img.shape
    ret = np.zeros([shape[0],shape[1],3])
    for i in range(0,3):
        ret[..., i] = img
        
    return ret

In [13]:
def rgb2Lanes(img):

    
    # Three Channel Color -> One Channel Gray
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    edges = getEdges(gray)
    
    imshape = img.shape[:2][::-1]
    # Mask Vertices Assuming A 1x1 Square
    generalVertices = np.array([[[.15,.92],[.45,.6],[.55,.6],[.85,.92]]])
    # Convert General Vertices In The Video's Resolution
    vertices = np.int32(np.multiply(generalVertices ,np.tile(imshape,(4,1))))

        
    masked_edges = maskImg(edges,vertices)
    
    lines_edges = getLines(masked_edges,edges)
    lines_overlayed = overlayLines(img,lines_edges)
    ret = lines_overlayed
    # One Channel Gray -> Three Channel Gray
    # ret = grayToThreeChannel(ret)
    
    return ret

In [14]:
new_clip_output = 'test_outputred.mp4'
test_clip = VideoFileClip("test.mp4")
new_clip = test_clip.fl_image(rgb2Lanes) 
%time new_clip.write_videofile(new_clip_output, audio=False)

[MoviePy] >>>> Building video test_outputred.mp4
[MoviePy] Writing video test_outputred.mp4


100%|█████████▉| 681/682 [00:27<00:00, 24.76it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_outputred.mp4 

CPU times: user 12.4 s, sys: 2.98 s, total: 15.4 s
Wall time: 28.7 s


In [15]:
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(new_clip_output))